<div style="background-color: lightblue; padding: 30px;">
    
# Digital House - Data Science a Distancia

## Proyecto Final Integrador

### Autores: Daniel Borrino, Ivan Mongi, Jessica Polakoff, Julio Tentor
    
<p style="text-align:right;">Agosto 2022</p>

---
</div>    

---
## Objetivo del Trabajo Final Integrador

<h4 style="text-align:center;">Demostrar que aprendimos conceptos y desarrollamos habilidades para obtener un modelo de aprendizaje automático que razonablemente predice un valor</h4>

---
### Contexto

???
---
### Requisitos del Trabajo Final Integrador

- Utilizar modelos de regresión o clasificación (es opcional)
- Evaluar modelos mediante ensamble, bagging o boosting
- Realizar selección de hiperparámetros
- Implementar pipeline al menos con algunos pasos del preprocesamiento
- Analizar importancia de características
- Serializar el modelo con mejor desempeño

---
### Entregables del Trabajo Final Integrador
#### Técnico (Jupyter Notebook)

1. Introducción y planteo del problema
2. Técnicas a utilizar y detalle del conjunto de datos
3. Desarrollo del análisis, visualizaciones, resultados de modelos
4. Detalle de hallazgos, conclusiones y recomendaciones

#### Presentación (audiencia NO TÉCNICA)

1. Resumen del problema
2. Métodos utilizados
3. Principales resultados

---
### Aspectos técnicos

La notebook se ejecuta correctamente en una instalación estándar de Anaconda versión *4.13.0 build  3.21.6, Python 3.9.7*

---

---

#### Necessary libraries


In [1]:
import pandas as pd
import numpy as np


#### Style

---

### Useful functions


---



---
<a id="Dataset"></a>

### Conjunto de datos - Dataset

Los datos originales son de [Manhattan Chocolate Society](http://flavorsofcacao.com/mcs_index.html)

El conjunto de datos se obtuvo de la [competencia en Datacamp](https://app.datacamp.com/learn/competitions/chocolate-bar-ratings) finalizada el 30 de junio de 2022.

---

*Las características en el conjunto de datos original son*:

    - id                   : Identificador de la revisión de la barra de chocolate
    

*Codificación de ingredientes en la barra de chocolate*

    - B  : Beans (grano)

*Escala de clasificación de una barra de chocolate*

    - 1.0  a 1.9  : Unpleasant (Desagradable)

---


---
<a id="Exploratory_data_analysis_01"></a>
### Exploración y análisis de datos básico del Conjunto de datos - Dataset
---


---
#### Observaciones sobre las características con valores nulos

- Las características 'ingredients' y 'num_ingredients' corresponden a las mismas observaciones, de modo que 'ingredients' se completa con el **valor más frecuente** y 'num_ingredients' que corresponde al **número de valores diferentes de la característica 'ingredients'** se completa con dicho valor

---

---
#### Observaciones

- Los valores de clasificación (raiting) se categorizan entre **menores y mayores o iguales a la moda**

---

---
#### Observaciones

- Las características 'year_reviewed' e 'id' están **significativamente relacionadas** con un nivel de confianza del 95%
- Es posible ignorar 'id'

---

---
#### Observaciones

- La característica 'manufacturer' indica muchas compañías, al rededor de un quinto de las observaciones; **no parece ser representativo**
- Las características 'company_location' y 'bean_origin' indican el país de la compañía y orígen del grano de cocoa; **¿son las mismas o diferentes?, ¿tienen alguna relación entre ellas y/o el valor de clasificación?**
- La característica 'ingredients' presenta 21 combinaciones diferentes; **¿todas son representativas?, ¿existe una relación particular con el valor de clasificación?**
---

---

---

---
### Evaluate base line models
---

---
#### Observaciones *algo para decir ...*

- ???

---

---

---



---
